In [6]:
import pandas as pd
import numpy as np
import os
import rebound

path = '/scratch/dtamayo/'
fcpath = path +'random/final_conditions/runs/fc'
shadowfcpath = path +'random/final_conditions/shadowruns/fc'
df = pd.read_csv(path+'random/random.csv', index_col=0)
df.tail()

,runstring
24995,0024995.bin
24996,0024996.bin
24997,0024997.bin
24998,0024998.bin
24999,0024999.bin


In [8]:
def get_times(row):   
    sim = rebound.Simulation.from_file(fcpath+row["runstring"])
    shadow = rebound.Simulation.from_file(shadowfcpath+row["runstring"])
    columns = ['t', 't_shadow']
    features = [sim.t, shadow.t]
    return pd.Series(features, index=columns)    

In [9]:
%%time
df = pd.concat([df, df.apply(get_times, axis=1)], axis=1)

/mnt/raid-cita/dtamayo/stability2/lib/python2.7/site-packages/rebound/simulation.py:374: RuntimeWarning: You have to reset function pointers after creating a reb_simulation struct with a binary file.
  warnings.warn(message, RuntimeWarning)


CPU times: user 11.9 s, sys: 802 ms, total: 12.7 s
Wall time: 14.6 s


In [10]:
df.head()

,runstring,t,t_shadow
0,0000000.bin,1.235644e+07,8.315081e+07
1,0000001.bin,1.348749e+05,2.133072e+05
2,0000002.bin,1.276381e+04,9.321519e+03
3,0000003.bin,1.000000e+09,1.000000e+09
4,0000004.bin,8.791888e+01,8.791888e+01


In [11]:
df.to_csv('../csvs/deltatimes.csv', encoding='ascii')